<a href="https://colab.research.google.com/github/arfin-arif/ml/blob/main/bangladigits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !unzip "/content/bengali_digits.zip" -d "/content/digit_images"

In [4]:
import csv
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
dataset = ImageDataGenerator(rescale=1/255)
train_dataset = dataset.flow_from_directory(
    '/content/digit_images',
    target_size=(28, 28),
    batch_size=32,
    class_mode='categorical',
    classes=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
)


Found 15620 images belonging to 10 classes.


In [9]:
#  Get the number of samples in your dataset
num_samples = train_dataset.samples

# Initialize lists to store pixel values and labels
pixel_values_list = []
labels_list = []

# Set batch size
batch_size = 32

# Iterate over batches to get all pixel values and labels
for _ in range(num_samples // batch_size):
    batch = next(train_dataset)
    images, labels = batch

    # Convert images to single-channel (grayscale) and flatten
    flattened_images = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).flatten() for image in images])

    # Extend the lists with current batch pixel values and labels
    pixel_values_list.extend(flattened_images)
    labels_list.extend(np.argmax(labels, axis=1).tolist())

# Create a CSV file and write pixel values and labels
csv_filename = 'dataset.csv'
with open(csv_filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write header
    header = ['Label'] + [f'{i}x{j}' for i in range(1, 29) for j in range(1, 29)]
    csvwriter.writerow(header)

    # Write data
    for pixel_values, label in zip(pixel_values_list, labels_list):
        row = [label] + pixel_values.tolist()
        csvwriter.writerow(row)

print(f"CSV file '{csv_filename}' created successfully.")

CSV file 'dataset.csv' created successfully.


In [4]:
import pandas as pd
df= pd.read_csv("/content/dataset.csv")
df

,Label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,2,1.000000,0.972549,0.976471,0.996078,0.996078,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,0.996078,1.000000,1.000000,0.992157,0.015686,0.007843,0.207843,0.988235
1,7,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.996078,0.992157,0.945098,0.011765,0.000000,0.117647,1.000000,0.992157,0.996078,0.996078
2,6,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.992157,1.000000,1.000000,1.000000,1.000000,1.000000,0.996078,1.000000,1.000000,1.000000
3,5,1.000000,1.000000,1.000000,0.996078,0.996078,1.000000,1.000000,1.000000,1.000000,...,0.172549,0.666667,0.650980,0.976471,0.988235,0.984314,1.000000,0.992157,0.996078,1.000000
4,5,1.000000,1.000000,0.996078,1.000000,0.988235,1.000000,0.980392,1.000000,0.980392,...,0.003922,0.203922,0.937255,0.964706,0.996078,0.996078,0.996078,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15583,1,1.000000,1.000000,1.000000,0.996078,0.992157,1.000000,1.000000,1.000000,0.996078,...,0.058824,0.745098,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
15584,9,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.988235,0.992157,1.000000,...,1.000000,1.000000,0.992157,0.996078,0.023529,0.007843,0.941177,1.000000,0.996078,1.000000
15585,2,1.000000,0.996078,1.000000,0.988235,0.996078,0.996078,1.000000,1.000000,0.968628,...,0.070588,0.019608,0.003922,0.003922,0.000000,0.003922,0.000000,0.023529,0.011765,0.003922
15586,9,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.996078,...,1.000000,0.988235,1.000000,0.572549,0.305882,0.290196,0.721569,1.000000,0.968628,0.992157


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
# Extract features (pixel values) and labels
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Train a Logistic Regression model
model = LogisticRegression(max_iter=1000,solver='liblinear',multi_class='ovr')
model.fit(X_train, y_train)

# Predict on the testing set
y_pred = model.predict(X_test)

# # Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 85.41%


In [8]:
y_test[:10]

array([3, 5, 4, 4, 3, 4, 3, 7, 0, 6])

In [12]:
row_to_test =4
true_label = y_test[row_to_test]
features_to_test = X_test[row_to_test, :].reshape(1, -1)
predicted_label = model.predict(features_to_test)[0]

print(f"True Label: {true_label}")
print(f"Predicted Label: {predicted_label}")


True Label: 3
Predicted Label: 3


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [14]:
svm = SVC()
svm.fit(X_train,y_train)
svm.score(X_test,y_test)

0.915009621552277

In [16]:
rf = RandomForestClassifier(n_estimators=40)
rf.fit(X_train,y_train)
rf.score(X_test,y_test)

0.9121231558691469

In [15]:
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

In [18]:
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=5)

scores_logistic = []
scores_svm = []
scores_rf = []

for train_index, test_index in folds.split(X,y):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], \
                                      y[train_index],y[test_index]
    scores_logistic.append(get_score(LogisticRegression(solver='liblinear',multi_class='ovr'), X_train, X_test, y_train, y_test))
    scores_svm.append(get_score(SVC(gamma='auto'), X_train, X_test, y_train, y_test))
    scores_rf.append(get_score(RandomForestClassifier(n_estimators=40), X_train, X_test, y_train, y_test))

In [20]:
print(scores_logistic )
print(scores_svm)
print(scores_rf)

[0.8422065426555484, 0.8543938422065427, 0.8508659397049391, 0.8466474173885146, 0.9076034648700674]
[0.8646568313021168, 0.8784477228992944, 0.8797305965362412, 0.8723131215912736, 0.8925248636509464]
[0.8986529826812059, 0.9108402822322001, 0.9124438742783836, 0.9024703240295155, 0.9736926531921719]


In [21]:
from sklearn.model_selection import cross_val_score

In [27]:
cross_val_score(LogisticRegression(solver='liblinear',multi_class='ovr'), X, y,cv=5)

array([0.84220654, 0.85439384, 0.85086594, 0.84664742, 0.90760346])

In [28]:
cross_val_score(SVC(gamma='auto'), X, y,cv=5)

array([0.86465683, 0.87844772, 0.8797306 , 0.87231312, 0.89252486])

In [26]:
cross_val_score(RandomForestClassifier(n_estimators=40),X, y,cv=5)

array([0.89448364, 0.91051956, 0.90859525, 0.90696182, 0.97273019])

### **Parameter Tuning for RF**

In [35]:
import numpy as np

In [36]:
s1=cross_val_score(RandomForestClassifier(n_estimators=20),X,y,cv=5)
np.average(s1)

0.9110239056114505

In [37]:
s2=cross_val_score(RandomForestClassifier(n_estimators=30),X,y,cv=5)
np.average(s2)

0.9164769622935165

In [38]:
s3=cross_val_score(RandomForestClassifier(n_estimators=30),X,y,cv=5)
np.average(s3)

0.9154503547040654

In [39]:
s4=cross_val_score(RandomForestClassifier(n_estimators=40),X,y,cv=5)
np.average(s4)

0.9196201467546528

In [40]:
s5=cross_val_score(RandomForestClassifier(n_estimators=50),X,y,cv=5)
np.average(s5)

0.9223144283361557

In [41]:
s6=cross_val_score(RandomForestClassifier(n_estimators=60),X,y,cv=5)
np.average(s6)

0.9232123987246993

In [42]:
s7=cross_val_score(RandomForestClassifier(n_estimators=80),X,y,cv=5)
np.average(s7)

0.9234691792386842

In [43]:
s8=cross_val_score(RandomForestClassifier(n_estimators=500),X,y,cv=5)
np.average(s8)

0.9273180882507583

In [44]:
s9=cross_val_score(RandomForestClassifier(n_estimators=2000),X,y,cv=5)
np.average(s9)

0.9286011882529603